In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import torch
from data_generation import signal, matrix, depth_model

<h1 style="text-align: center; margin-left: 100px; margin-right: 100px">
Estimation de la profondeur des téléséismes par apprentissage automatique à partir de signaux synthétiques
</h1>

<p style="text-align: justify;">
La caractérisation de la profondeur d'un évènement sismique est un sujet important parmis les études menées sur les séismes : cela peut-être un critère de discrimination entre plusieurs types d'évènements (Laporte, 2022), ou encore simplement apporter des informations sur la nature géologique d'une zone d'étude. La densification et le renforcement de la couverture azimutale des réseaux sismologiques durant les dernières décennies permettent à présent d’optimiser la récupération pour un évènement de l’énergie des phases dites « de profondeur » (phases pP et sP), parfois détectables seulement dans certains azimuts. Ces phases permettent de déduire la profondeur de la source d’un évènement à distance télésismique (de 3000 à 10 000 km).
</p>

<p style="text-align: justify;">
Des méthodes récentes (p.ex. Blackwell et al, 2024) se portent sur une sommation de nombreux signaux télésismiques cohérents afin de maximiser l’énergie de ces phases par rapport au bruit, puis sur une double migration en profondeur selon chacune des deux hypothèses (pP et sP), permettant de co-valider les résultat de profondeur obtenus. L'objectif de ce projet est d'estimer la profondeur de téléséismes directement à partir des signaux, ou plus particulièrement à partir d'une matrice correspondant à l'enveloppe énergétique de ces signaux (v. <strong>Figure 1</strong>), sans passer par cette étape de migration en profondeur qui peut être un peu lourde en calculs, en développant un modèle d'apprentissage automatique. Par manque de données facilement exploitables, le modèle a été entraîné sur des données synthétiques dont la génération est décrite ci-après.
</p>

<figure>
    <img src="notebook_images/real_energetic_envelope.png" style="width: 750px; display: block; margin: auto;" alt="Exemple des enveloppes énergétiques pour un évènement réel">
    <figcaption style="text-align: center;"><strong>Figure 1 :</strong> Enveloppes énergétiques pour un évènement sismique réel de magnitude 5, le 24 mai 2024 à Zabaykal'ye, Russie (Dupont, 2024)</figcaption>
</figure>



## Génération des synthétiques

<p style="text-align: justify;">
Il s'agit de générer, pour un évènement aléatoire d'une profondeur maximale de 100 km (fixée arbitrairement), les signaux captés en 50 stations à distance télésismique, choisies aléatoirement également. Le modèle convolutif sera ensuite entraîné à partir de ces données pour plusieurs évènements. Les modules nécessaires pour la génération des signaux sont <strong>math</strong>, <strong>random</strong>, <strong>obspy</strong>, <strong>numpy</strong>, <strong>matplotlib</strong>, <strong>scipy</strong>, ainsi que <strong>pytorch</strong> pour la génération des matrices sous forme de tenseurs compatibles avec l'entraînement du modèle.
</p>

#### Générer un signal unique

<p style="text-align: justify;">
On utilise le modèle de Terre 1D simplifié <strong>ak135</strong> (Kennett, 2005) disponible dans TauP pour calculer les temps de trajets - et donc les arrivées - des ondes P, pP et sP à partir de coordonnées aléatoires pour la source (latitude, longitude, profondeur) et les stations (latitude et longitude, obligatoirement à distance télésismique par rapport à la source).
</p>

<p style="text-align: justify;">
Ensuite, on génère un signal de 60 s pour chacune de ces stations à partir des délais entre les arrivées des différentes phases, en plaçant un dirac pour chaque arrivée. Pour augmenter le réalisme du signal, la décroissance énergétique exponentielle des phases est simulée par l'ajout de diracs à intervalle régulier, d'amplitude de moins en moins importante. L'amplitude des phases est aléatoire (entre 0.5 et 1 pour la phase P, et de 0 à 110% de la phase P pour chacune des phases de profondeur), ainsi que leur signe original (qui peut également basculer), dans le but de simuler le mécanisme à la source. Une inversion du signe également possible, pour tenter de simuler des inversion de signe après interaction avec certaines interfaces. La génération des signaux est largement améliorable pour rendre les synthétiques plus réalistes, en particulier sur la manière de simuler la radiation à la source ainsi que la propagation des signaux.
</p>

<p style="text-align: justify;">
Ces signaux discrets sont ensuite convolués avec une ondelette de Ricker afin de générer un signal continu, auquel on ajoute du bruit blanc - avec un ratio signal sur bruit aléatoire compris entre 2 et 5 - avant de le filtrer entre 0.8 et 2.5 Hz (Laporte, 2022). Ce signal est enfin normalisé par Z-score (pour annuler le phénomène de poids lié à la décroissance énergique des signaux avec la distance) avant d'en extraire l'enveloppe de Hilbert absolue, qui est elle-même normalisée par le maximum (dans le but d'unifier les amplitudes de tous les signaux). Cette dernière normalisation des signaux est rendue possible par le fait qu'on ne s'intéresse qu'à leur forme lorsqu'on cherche à mettre en évidence les phases de profondeur. Les signaux sont ensuite décimés à 20 Hz, largement au-dessus de la fréquence de Nyquist ($ f_{\text{N}} = $ 5 Hz).
</p>

In [ ]:
signal, source, stations = signal.generate_one_signal(plot=True)

#### Générer une matrice unique

<p style="text-align: justify;">
L'étape de génération d'un signal est répétée 50 fois afin de produire une matrice de dimensions (50, 1200) pour un même évènement. La profondeur est gardée en mémoire pour le calcul du misfit lors de l'entraînement - ainsi que les distances des stations à l'épicentre, pour éventuellement fournir plus d'informations au modèle. Pour pouvoir être utilisées lors de l'entraînement de l'algorithme, ces distances sont normalisées par min/max avant d'être sauvegardées, par suppression du minimum possible (fixé à 2500 km) et division par l'écart entre le minimum et le maximum (fixé à 10000 km).
</p>

In [ ]:
signal_matrix, depth, distances = matrix.generate_matrix(num_stations=50, plot=True)

## Entraînement du modèle d'apprentissage automatique

#### Etape d'entraînement

<p style="text-align: justify;">
L'entraînement se fait à partir d'un batch <strong>X</strong>. Ce batch est un set de données sous forme d'un tenseur de dimensions (256, 1, 50, 1200), correspondant aux données de 256 évènements en niveaux de gris, avec 50 signaux de longueur 1200 pour chaque évènement. La taille de batch (256) est adaptable, mais le modèle le plus performant a été entrainé sur un batch de 256 évènements. Les profondeurs (nécessaires pour le calcul du misfit) sont enregistrées comme un tenseur <strong>y</strong> de dimensions (256, 1), et les distances à la source comme un tenseur <strong>D</strong> de dimensions (256, 50). Les modules nécessaires pour l'entraînement du modèle sont <strong>pytorch</strong>, <strong>optuna</strong>, <strong>tqdm</strong>, <strong>numpy</strong> et <strong>matplotlib</strong>.
</p>

<p style="text-align: justify;">
Le modèle compte différentes couches, d'abord convolutives pour récupérer les informations spatiales et temporelles à différentes échelles contenues dans les signaux, puis pleinement connectées - afin de réduire la taille de sortie progressivement à une seule profondeur. Les différentes couches sont les suivantes :
<ul>
<li> une première convolution avec 1 canal d'entrée et 16 canaux de sortie, un kernel de taille (3, 3) et un padding correspondant (1, 1). Elle est suivie d'une normalisation 2D et d'un pooling maximal avec un kernel de taille (2, 1), pour réduire la dimension spatiale (nombre de stations) sans réduire la dimension temporelle (longueur du signal) ;</li>
<li> une deuxième convolution similaire avec 16 canaux d'entrée et 32 canaux de sortie, et les même étapes de normalisation et de pooling maximal ;</li>
<li> une dernière convolution similaire avec 32 canaux d'entrée et 64 canaux de sorties, et également les même étapes de normalisation et de pooling maximal que précédemment ;</li>
<li> un applatissement des sorties convolutionelles en un seul vecteur, pour préparer aux couches pleinement connectées ;</li>
<li> la concaténation de ce vecteur avec la dimension de <strong>D</strong> contenant les distances à la source (qui correspond à un ajout de caractéristiques pour mieux contraindre le modèle) ;</li>
<li> une couche linéaire de 512 unités avec dropout de 20 %, pour limiter l'overfitting ;</li>
<li> une couche linéaire de 256 unités avec dropout de 20 % ;</li>
<li> une couche linéaire de 128 unités avec dropout de 20 % ;</li>
<li> une couche linéaire de 64 unités avec dropout de 20 % ;</li>
<li> une dernière couche de sortie linéaire avec 1 unité correspondant à la profondeur.</li>
</ul>
</p>

<p style="text-align: justify;">
Enfin, l'entraînement du modèle se fait de la manière suivante :
<ul>
<li> génération des ensembles de données d'entraînement (<strong>X_train</strong>, <strong>y_train</strong>, <strong>D_train</strong>), de validation (<strong>X_val</strong>, <strong>y_val</strong>, <strong>D_val</strong>) et de test (<strong>X_test</strong>, <strong>y_test</strong>, <strong>D_test</strong>) à partir de profondeur uniformément réparties de 0 à 100 km de profondeur, pour entraîner le modèle sur l'entièreté de la gamme de profondeur ;</li>
<li> mélange des ensembles de données, puisqu'elles sont générées par ordre croissant de profondeur ;</li>
<li> initialisation du modèle sur le GPU, puis optimisation des hyperparamètres avec <strong>optuna</strong> : le taux d'apprentissage et le poids de dégradation pour l'optimiseur, ainsi que le pas et le gamma de l'agenda ;</li>
<li> initilisation de l'optimiseur Adam - qui semble être reconnu pour son utilité lors de tâches complexes dans les géosciences (Arouri et Sayyafzadeh, 2022) et pour son efficacité dans le traitement de signaux sismiques en apprentissage profond (Richardson, 2018) - et de l'agenda (StepLR) à partir des meilleurs solutions d'hyperparamètres ;</li>
<li> initialisation de la fonction de perte MAE (Mean Absolute Error) qui, contrairement à la MSE (Mean Squared Error), réduis les biais systématiques plutôt que de se concentrer sur les zones difficiles à prévoir, permettant ainsi une meilleure généralisation (mais en revanche permet une erreur plus importante dans certains cas) ;</li>
<li> entraînement du modèle sur 200 époques avec calculs de la perte, ajustement des poids par rétropropagation, ajustement du taux d'apprentissage par l'agenda, et validation après chaque époque ;</li>
<li> évaluation du modèle sur l'ensemble de test pour obtenir la perte de test finale ;</li>
<li> sauvegarde du modèle entraîné (c'est-à-dire des poids, qui déterminent comment le modèle prédit la donnée de sortie à partir de celles d'entrée).</li>
</ul>

In [ ]:
model_name, train_losses, val_losses, test_loss = depth_model.train_DepthModel(model_name="Final_Model",
                                                                               batch_size=256,
                                                                               num_stations=50,
                                                                               epochs=200,
                                                                               include_distance=True,
                                                                               plot=True
                                                                              )

#### Résultats du modèle

In [ ]:
real_depth, predicted_depth, delta_depth = depth_model.test_DepthModel(model_name="Final_Model",
                                                                       num_test=1000,
                                                                       num_stations=50,
                                                                       include_distance=True
                                                                      )

#### Utiliser un modèle existant

In [ ]:
delta_depth = depth_model.run_random_DepthModel(model_name="Final_Model",
                                                device_name="cuda",
                                                num_stations=50,
                                                include_distance=True,
                                                depth_list=[50*1e3],  # profondeur (m)
                                                plot=True
                                               )

<div style="font-family: Arial, sans-serif; font-size: 14px;">
    <h2>Bibliographie</h2>
    <ul>
        <li>Arouri, Y. et Sayyafzadeh, M. : <cite>An adaptive moment estimation framework for well placement optimization</cite>, Computational Geosciences, 26, 957–973, <a href="https://doi.org/10.1007/s10596-022-10135-9">https://doi.org/10.1007/s10596-022-10135-9</a>, 2022.</li>
        <li>Blackwell et al., 2024</li>
        <li>Dupont, B. : <cite>Évaluation de nouvelles méthodes de discrimination d'évènements sismiques par analyse des phases de profondeur</cite>, rapport de stage, 2024.</li>
        <li>Kennett, B. L. N. : <cite>AK135 Tables</cite>, <a href="https://www.researchgate.net/publication/257584026_AK135tables">https://www.researchgate.net/publication/257584026_AK135tables</a>, 2005.</li>
        <li>Paszke, A., Gross, S., Massa, F., et al. : <cite>PyTorch: an imperative style, high-performance Deep Learning library</cite>, <a href="https://doi.org/10.48550/arXiv.1912.01703">https://doi.org/10.48550/arXiv.1912.01703</a>, 2019.</li>
        <li>Laporte, M. : <cite>Contribution à l'amélioration de l'estimation de la profondeur hypocentrale à partir de réseaux régionaux ou globaux</cite>, thèse de doctorat, Ecole Normale Supérieure, <a href="https://theses.hal.science/tel-04635638">https://theses.hal.science/tel-04635638</a>, 2022.</li>
        <li>Richardson, A. : <cite>Seismic Full-Waveform Inversion using Deep Learning tools and techniques</cite>, <a href="https://doi.org/10.48550/arXiv.1801.07232">https://doi.org/10.48550/arXiv.1801.07232</a>, 2018.</li>
    </ul>
</div>
